In [12]:
%load_ext autoreload
%autoreload 2

import sys, os, re
print(sys.version)

import random
from pprint import pprint
from time import sleep
import math
from itertools import groupby
from collections import OrderedDict as odict
from collections import defaultdict
from collections import namedtuple
from collections import Counter
import yaml
import json
import io
from datetime import datetime
import platform
import fnmatch
import inspect
from itertools import compress
from itertools import cycle
from collections import deque
import functools


from pathlib import Path

def datalines(text, limit=None, comment='#'):
  '''返回一段文字中有效的行(非空行, 且不以注释符号开头)'''
  lines = [line.strip() for line in text.splitlines()]
  lines = [line for line in lines if line and not line.startswith(comment)]
  if limit:
    return lines[:limit]
  else:
    return lines

def findone(lst, cond, default=None):
  '''选择列表中第一个匹配的元素'''
  return next((x for x in lst if cond(x)), default)

def dedupe(items, key=None):
  '''去除重复，可以保留原list的顺序'''
  seen = set()
  for item in items:
    val = item if key is None else key(item)
    if val not in seen:
      yield item
      seen.add(val)


def windows(iterable, length=2, overlap=0, yield_tail=True):
  '''按照固定窗口大小切片list, 可以重叠
  滑动array窗口,
  每次提供length数目的元素,如果有overlap则重复之前的元素
  yield_tail: 最后不足 length 的那部分元素是否也要 yield'''
  import itertools
  if length <= overlap:
    raise AttributeError(
        'overlap {} cannot larger than length {}'.format(overlap, length))
  it = iter(iterable)
  results = list(itertools.islice(it, length))
  while len(results) == length:
    yield results
    results = results[length-overlap:]
    results.extend(itertools.islice(it, length-overlap))
  if results and yield_tail:
    yield results


import json
def json_load(path):
  with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)
  return data
def json_save(path, data):
  with open(path, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
def json_loads(s):
  return json.loads(s)
def json_saves(data):
  return json.dumps(data)



import os
def text_load(path, encoding=None):
  ''' 读取文本, 尝试不同的解码 '''
  if not os.path.exists(path):
    raise ValueError("path `{}` not exist".format(path))
  if encoding is None:  # 猜测 encoding
    try:
      open(path, 'r', encoding='utf-8').read()
      encoding = 'utf-8'
    except UnicodeDecodeError:
      encoding = 'gbk'
  with open(path, 'r', encoding=encoding) as f:
    ret = f.read()
  return ret
def text_save(path, data, encoding='utf-8'):
  with open(path, 'w', encoding=encoding) as f:
    f.write(data)
  return True




"""--------------
同时 log 到 stdout 和 file.log
--------------"""
from pprint import pprint
import datetime

class create_logger:
  def __init__(self, file_path, pretty_format='print'):
    # pretty_format = pprint print brief
    self.filepath = file_path + '.log'
    self.pretty_format = pretty_format

  def brief_format(self, data):
    # 简略处理 str list dict 等 # 认为此时不会有复杂对象
    if isinstance(data, str):
        if len(data) < 300:
            return data
        else:
            return data[:240] + ' ... ' + data[-50:] + f' ({len(data)} chars)'
    if isinstance(data, list):
        if len(data) < 20:
            return data
        else:
            return data[:10] + [f' ... (total {len(data)} items) ... ' ] + data[-5:]
    if isinstance(data, dict):
        length = len(data.keys())
        if length < 20:
            return data
        else:
            return '\n'.join(f'({i}) {k}: {v}' for i, (k, v) in enumerate(
                data.items()) if (i < 10 or i + 5 > length)
                ) + f' (total {length} keypairs)'
    # 其余默认不处理
    return data

  def custom_print(self, data, prefix='', filepath=None, pretty=None):
    pretty = pretty or self.pretty_format
    out = open(filepath, 'a', encoding='utf-8') if filepath else sys.stdout
    if filepath:  # 在输出到文件时增加记录时间戳, 输出到 stdout 不记录时间戳
      prefix = '[' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + ']' + prefix

    if prefix:
      print(prefix, file=out, end=' ')

    if pretty == 'pprint':
      pprint(data, stream=out, width=120, compact=True, depth=3, sort_dicts=False)
    elif pretty == 'brief':
      print(self.brief_format(data), file=out)
    else:
      print(data, file=out)
    # if filepath:
    #   out.close()
  def output(self, data, pretty):
    # 首先输出到控制台, 其次文件
    try:
      self.custom_print(data, filepath=None, pretty=pretty)
    except UnicodeEncodeError as e:
      self.custom_print(str(e), prefix='logger output error: ')
    try:
      self.custom_print(data, filepath=self.filepath, pretty=pretty)
    except UnicodeEncodeError as e:
      self.custom_print(str(e), filepath=self.filepath, prefix='logger output error: ')

  def __ror__(self, other):  # 还是先改成只接受一个参数
    self.output(other, self.pretty_format)
    return other
  def __call__(self, other, pretty='pprint'):
    self.output(other, pretty=pretty)
    return other



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [13]:
loge = create_logger('tmpl' + '.error')
logi = create_logger('tmpl', pretty_format='brief')  # brief print

sample = list(range(30))

sample | logi
print('-----')
print(sample)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, ' ... (total 30 items) ... ', 25, 26, 27, 28, 29]
-----
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
